<a href="https://colab.research.google.com/github/Ksenia-90/NN/blob/lesson_8/dz_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Урок 8. GAN

Добрый день! Дополнила раздор статьи из 7 задания (и отнесла ее и к 8 уроку в более расширенном разборе)

Point-GNN: Graph Neural Network for 3D Object Detection in a Point Cloud: https://openaccess.thecvf.com/content_CVPR_2020/papers/Shi_Point-GNN_Graph_Neural_Network_for_3D_Object_Detection_in_a_CVPR_2020_paper.pdf

В данной статье описывается 3D моделирование. В статье эксперты утверждают, что контрольные показатели показывают, что предлагаемый подход достигает лидирующих точность с использованием только облака точек и может даже превзойти алгоритмы на основе слияния, демонстрируют потенциал использования графовой нейронной сети как нового подхода к обнаружению 3D-объектов. Loss oписан в пункте 3.3 Oперация свертки эффективна, но для нее требуется регулярная сетка, В отличие от изображения, облако точек обычно является разреженным и неравномерно распределены по регулярной сетке. Учеными, в статье, предполается что данная сеть будет использоваться для высокоточного обнаружения обьектов. В слое подачи данных самая большая проблема - подаётся граф на основании данных лидара. Point-GNN принимает точечный график в качестве входных данных. Выводит категорию и ограничивающие рамки объектов, которым принадлежит каждая вершина. В статье дается ссылка на код: https://github.com/WeijingShi/Point-GNN. В тесте KITTI Point-GNN достигает высочайшая точность с использованием только облака точек и даже превосходит подходы слияния датчиков. Точка-GNN демонстрирует потенциал подхода к обнаружению 3D-объектов нового типа с использованием графовой нейронной сети, и он может служить в качестве хорошая база для будущих исследований. Проводитлись обширные абляционные исследование эффективности компонентов Point-GNN. В этой статье: • Мы предлагаем новый подход к обнаружению объектов с использованием граф нейронной сети на облаке точек. • Мы разрабатываем Point-GNN, графовую нейронную сеть с механизм автоматической регистрации, который обнаруживает несколько объектов в одном кадре. • Мы достигаем высочайшей точности обнаружения 3D-объектов в тесте KITTI и тщательно анализируем эффективность каждого компонента.
* общая архитектура  метода состоит из трех компонентов: построение графа, (GNN из T итераций, и  слияние ограничивающих рамок и подсчет очков.
Построением такого графа является задача поиска ближайших соседей по радиусу. 
После построения графа обрабатывается граф с помощью GNN.
Типичная графовая нейронная сеть уточняет функции вершин, агрегируя функции вдоль ребер. Если вершина находится внутри ограничивающей рамки объекта, мы присваиваем вершине класс объекта.
несколько вершин могут находиться на одном и том же объекте, нейронная сеть может выводить несколько ограничивающих прямоугольников одного и того же объекта.
объект. Необходимо объединить эти ограничивающие рамки в
один, а также присвоить показатель достоверностию
Набор данных KITTI содержит 7481 обучающая выборка и 7518 тестовых выборок. Каждый образец содержит как облако точек, так и изображение с камеры.
Необходимо удаление шума.
Используя графовое представление, можно
компактно закодировать облако точек без сопоставления с
сетки или выборки и группировки неоднократно. Предлагаемый механизм автоматической регистрации снижает дисперсию перехода, а слияние блоков и подсчет очков операция повышает точность обнаружения. 

In [1]:
import numpy as np
import pandas as pd

import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file = open('/content/drive/My Drive/Colab Notebooks/drakula_2.txt').read()

In [5]:
def tokenize_words(input):
    input = input.lower()

    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

In [6]:
processed_inputs = tokenize_words(file)

In [7]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))


In [8]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 31345
Total vocab: 37


In [9]:
seq_length = 100
x_data = []
y_data = []

In [10]:
for i in range(0, input_len - seq_length, 1):
    in_seq = processed_inputs[i:i + seq_length]

    out_seq = processed_inputs[i + seq_length]

    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])


In [11]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 31245


In [12]:
X = np.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [13]:
y = np_utils.to_categorical(y_data)

In [14]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.15))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.15))
model.add(LSTM(128))
model.add(Dropout(0.15))
model.add(Dense(y.shape[1], activation='softmax'))


In [15]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [16]:
model.fit(X, y, epochs=50, batch_size=256)

Epoch 1/50
123/123 [==============================] - 426s 3s/step - loss: 2.9740
Epoch 2/50
123/123 [==============================] - 431s 4s/step - loss: 2.9176
Epoch 3/50
123/123 [==============================] - 437s 4s/step - loss: 2.7999
Epoch 4/50
123/123 [==============================] - 419s 3s/step - loss: 2.6705
Epoch 5/50
123/123 [==============================] - 417s 3s/step - loss: 2.6236
Epoch 6/50
123/123 [==============================] - 418s 3s/step - loss: 2.5852
Epoch 7/50
123/123 [==============================] - 418s 3s/step - loss: 2.5493
Epoch 8/50
123/123 [==============================] - 421s 3s/step - loss: 2.5172
Epoch 9/50
123/123 [==============================] - 421s 3s/step - loss: 2.4849
Epoch 10/50
123/123 [==============================] - 424s 3s/step - loss: 2.4537
Epoch 11/50
123/123 [==============================] - 423s 3s/step - loss: 2.4256
Epoch 12/50
123/123 [==============================] - 427s 3s/step - loss: 2.3957
Epoch 13/50
1

In [18]:
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [19]:
start = np.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")


Random Seed:
" find home dawn struggling windows felt morning day overwork flesh answered pinching test eyes deceiv "


In [20]:
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = num_to_char[index]

    sys.stdout.write(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]


ed sall tent exen tindow see wind coule see shall said thin thndo lan taid shall tert pertens begind said taid taid taid thie went dear _bc whie went asay hoow sodt shil soom lonki lan said sha man lieht pat sare shall teen sent koonin thie woued shall ready sime shall tall thndo loneon seall tiies teneome hous windo ligh gate shough wind shall rtran hand shoug light strne tiie shough went soon little wiodow seeth shose shough whnse shoe fare tould soust exer eren whndow sould said soom looki lany said couet window sould said soom look dount ray sinugh shou shou hear seeth sent bed gear _oye sid soom lonki lan aartle went sent sent sent exer sindow seeth shose shou hear sout side coule couet went sent shen went rent eeas siies ploe larter sime larked could see shall nany tent eoead said thing said could see seen could see shall nany tineow bount eroan wiete sould sime shoug shoug shos hear shing sould sime wineow bea sti shou aske might strang hand tould said said thing said coacula se

Можно еще поэксперементировать с показателями. очень много раз не запускала, очень очень долго считается.